In [1]:
# Author: Ian Hothi
# Date: Feb 2025

import numpy as np
#For Power Spectrum Calculations 
import tools21cm as t2c

# About the data

The data has been simulated using 21cmFast. The XY plane has an extent of 250Mpc, with 256 pixels on each side. The line-of-sight (redshift) has been simulated to be between z = 8.82 (144.60 MHz) and z = 9.33 (137.46 MHz), comprising 128 frequency channels. The three parameters chosen to be changed are the virial temperature, maximum bubble size, and the ionising efficiency parameter—these were all found to have the strongest impact on the signal. 

These are the parameter variations (fiducial value $\pm$ change):

- $T_{Vir}$: $50000\pm 5000$
- $R_{Max}$: $15\pm 5 Mpc$
- $\zeta$: $30\pm 5$,
400 simulations were run for each parameter value, which will be used for the derivatives. In the loaded files, there will be two files for each parameter, one corresponding to the plus value - for example, for $T_{Vir}$ tue value would be 50000 + 5000. Whereas the minus value would be 50000 - 5000.


400 simulations were run with the fiducial values, this will serve to calculate the covariances of the statistics.




## Fisher Parameters

In [2]:
# Astro params
params = ['ION_Tvir_MIN']#['ION_Tvir_MIN','R_BUBBLE_MAX','HII_EFF_FACTOR']
nparams = len(params)
delta_params = [pow(10,4.740362689494244)-pow(10,4.653212513775344),10,10]

# Sim params
# Lightcone dimensions
box_dim = 256
box_length = 200

# Power spectrum params
nbins = 15


# Loading Data and Statistics estimation 

In [3]:
import os
import gc
import numpy as np

# Configuration
ddir      = '/SKA_Chapter_simulations/'
out_dir   = '/SKA_Chapter_simulations/PowerSpectra/'  # make sure this exists!
nbins     = 15
box_length= 400

# List of (filename, output_basename) pairs
jobs = [
    ('Lightcone_ION_Tvir_MIN_400_Samples_Plus.npz',     'PS_param_1_plus'),
    ('Lightcone_ION_Tvir_MIN_400_Samples_Minus.npz',    'PS_param_1_minus'),
    ('Lightcone_R_BUBBLE_MAX_400_Samples_Plus.npz',     'PS_param_2_plus'),
    ('Lightcone_R_BUBBLE_MAX_400_Samples_Minus.npz',    'PS_param_2_minus'),
    ('Lightcone_HII_EFF_FACTOR_400_Samples_Plus.npz',   'PS_param_3_plus'),
    ('Lightcone_HII_EFF_FACTOR_400_Samples_Minus.npz',  'PS_param_3_minus'),
    ('Window_Functions/Noiseless/Lightcone_FID_400_Samples.npz', 'PS_fid'),
]

for fname, out_base in jobs:
    path = os.path.join(ddir, fname)
    print(f'Processing {fname}…')
    
    # Memory‐map the huge array
    data = np.load(path, mmap_mode='r')['arr_0']
    n_samp = data.shape[0]
    
    # Prepare output container
    PS  = np.zeros((n_samp, nbins), dtype=np.float32)
    ks  = None
    
    # Loop over each sample
    for i in range(n_samp):
        PS[i], ks = t2c.power_spectrum_1d(
            data[i],
            kbins=nbins,
            box_dims=box_length
        )
        if i and i % 50 == 0:
            print(f'  … {i}/{n_samp} done')
    
    # Save just the power spectra + k‐bin centers
    out_path = os.path.join(out_dir, out_base + f'_{nbins}bins.npz')
    np.savez_compressed(out_path, PS=PS, ks=ks)
    print(f'  Saved → {out_path}')
    
    # Teardown to free memory
    del data, PS
    gc.collect()

Processing Lightcone_ION_Tvir_MIN_400_Samples_Plus.npz…


FileNotFoundError: [Errno 2] No such file or directory: '/SKA_Chapter_simulations/Lightcone_ION_Tvir_MIN_400_Samples_Plus.npz'

In [ ]:
import os
import numpy as np
import glob

# where you dumped the .npz outputs
stat_dir = '/SKA_Chapter_simulations/PowerSpectra/'

# grab all the 15-bin runs
files = glob.glob(os.path.join(stat_dir, '*_15bins.npz'))

# container dict: keys will be e.g. 'PS_param_1_plus', 'PS_fid', etc.
PS_data = {}

for fpath in files:
    # derive a clean name from the filename
    basename = os.path.basename(fpath).replace('_15bins.npz','')
    
    # load just once
    with np.load(fpath) as dd:
        PS   = dd['PS']    # shape (n_samples, nbins)
        ks   = dd['ks']    # shape (nbins,)
    
    PS_data[basename] = {
        'PS':  PS,
        'ks':  ks,
    }

# now you can refer to, e.g. PS_data['PS_param_1_plus']['PS'] 
# and PS_data['PS_param_1_plus']['ks'] etc.

# Example: check shapes
for name, dat in PS_data.items():
    print(f"{name:25s} → PS {dat['PS'].shape},  ks {dat['ks'].shape}")


### Fisher Matrix

Under the assumption that the likelihood is a multivariate Gaussian (mean vectors are sufficient information for the covariance matrix), the Fisher matrix is defined as:

$$F^\theta_{ij} = \frac{\partial \textbf{S}}{\partial\theta_i} \mathbf{\Sigma}^{-1} \frac{\partial \textbf{S}}{\partial\theta_j},$$

where  $\frac{\partial \textbf{S}}{\partial\theta_i}$ is the change in statistic S under a change in paramrter i of $\partial\theta_j$. Here, $\mathbf{\Sigma}$ is the the covariance matrix, given my the fiducial simulation set.


Let us first calculate $\frac{\partial \textbf{S}}{\partial\theta_i}$ for out three astrophysical parameters:

In [ ]:
import numpy as np

# 1) Pull the PS arrays out of your PS_data dict
PS1p = PS_data['PS_param_1_plus']['PS']
PS1m = PS_data['PS_param_1_minus']['PS']

PS2p = PS_data['PS_param_2_plus']['PS']
PS2m = PS_data['PS_param_2_minus']['PS']

PS3p = PS_data['PS_param_3_plus']['PS']
PS3m = PS_data['PS_param_3_minus']['PS']

ks   = PS_data['PS_param_1_plus']['ks']

# 3) Compute the finite‐difference derivatives for each k‐bin & sample
dPS1 = (PS1p - PS1m) / delta_params[0]
dPS2 = (PS2p - PS2m) / delta_params[1]
dPS3 = (PS3p - PS3m) / delta_params[2]

nparams  = 3
nsamples = PS1p.shape[0]
nbins    = PS1p.shape[1]

PS_derivs = np.zeros((nparams, nsamples, nbins))
PS_derivs[0] = dPS1
PS_derivs[1] = dPS2
PS_derivs[2] = dPS3



Let us whiten the data to get rid of any order of magnitude issues that may arise:

In [ ]:
## Whitening step ##
PS_fid = PS_data['PS_fid']['PS'] 

std_fid = np.std(PS_fid,axis=0)
white_fid = PS_fid/std_fid
#The covariance
covariance = np.cov(white_fid,rowvar=False)
# Whitening the derivatives
PS_derivs_white = PS_derivs.reshape((nparams,max_samples,-1))/std_fid

# To check the condition of the covariance:
print(('Condition Number: 10^%.3f')%(np.log10(np.linalg.cond(covariance))))

We can now calculate the Fisher Matrix

In [ ]:
# Initialising Fishers 
PS_Fisher = np.zeros((len(samples),nparams,nparams))
PS_Fisher_Inv = np.zeros((len(samples),nparams,nparams))

# How many samples to use
max_samples = len(PS_derivs_white[0])
samples = np.arange(5,max_samples+50,5)

In [ ]:
# Calculating the Fisher matrix for a given sample of derivatives 
for k,sample_size in enumerate(samples):
    deriv_sample = np.mean(derivatives_white[:,:sample_size,:],axis=1)
    F_ij = np.zeros((nparams,nparams))
    for i in range(nparams):
        for j in range(nparams):
            parm_i_ps = (deriv_sample)[i]
            parm_j_ps = (deriv_sample)[j]        
            FIJ = np.dot(parm_i_ps, np.dot(np.linalg.inv(covariance),parm_j_ps))
            F_ij[i,j] = FIJ
    F_ij_inv = np.linalg.inv(F_ij)  
    PS_Fisher[k,:,:]= F_ij
    PS_Fisher_Inv[k,:,:] = F_ij_inv
    


This is to be explicit, but we can just put this all into a function as:

In [ ]:
def get_fisher(derivatives,fiducial,nparams):
    """ Calculates the fisher matrix for different number of derivative samples. This function also 
        plots the 'identity' matrix derived for multiplying the covariance by its inverse - we also 
        include a histogram of this 'identity' matrix to see the zero-error.

    Parameters
    ----------
    derivatives : numpy.array
        An array containing the derivatives of a given statistic, with dimensions of 
        [nparams,nsamples,ncoefficients]
    fiducial : numpy.array
        Contains the fiducial simulation's statistic, with which we calculate the
        covariance. It has dimensions [nsamples,ncoefficients].
    nparams : int
        The number of parameters for which a derivative has been calculated.
    
    Returns
    -------
    Fisher : numpy.array
        Fisher matrix for a given number of sample used to calculate derivatives.
        Dimensions of [len(samples), nparams, nparams]
        
    Fisher_Inv : numpy.array
        Inverse of the Fisher matrix,having dimensions of [len(samples), nparams, nparams].
    samples : numpy.array
        Contains the number of samples of derivatives used to calculate Fisher and Fisher_Inv
    """
    import numpy
    # Maximum number of derivative samples that can be used 
    max_samples = len(derivatives[0])
    
    ## Whitening step ##
    std_fid = np.std(fiducial,axis=0)
    white_fid = fiducial/std_fid
    #The covariance
    covariance = np.cov(white_fid,rowvar=False)
    # Whitening the derivatives
    derivatives_white = derivatives.reshape((nparams,max_samples,-1))/std_fid
    print(('Condition Number: 10^%.3f')%(np.log10(np.linalg.cond(covariance))))
    # How many samples to use
    samples = np.arange(5,max_samples+50,5)
    # Initialising Fishers 
    Fisher = np.zeros((len(samples),nparams,nparams))
    Fisher_Inv = np.zeros((len(samples),nparams,nparams))
    
    
    # Checking stability of the covariance
    plt.figure()
    I = np.log10(abs(np.matmul(np.linalg.inv(covariance),covariance))+1e-16)
    plt.imshow(I)
    plt.colorbar().set_label('log$_{10} C^{-1}C$')
    plt.figure()
    plt.hist(I.reshape(-1),bins=100)
    plt.xlabel('log$_{10} C^{-1}C$')
    plt.yscale('log')

    # Calculating the Fisher matrix for a given sample of derivatives 
    for k,sample_size in enumerate(samples):
        deriv_sample = np.mean(derivatives_white[:,:sample_size,:],axis=1)
        F_ij = np.zeros((nparams,nparams))
        for i in range(nparams):
            for j in range(nparams):
                parm_i_ps = (deriv_sample)[i]
                parm_j_ps = (deriv_sample)[j]        
                FIJ = np.dot(parm_i_ps, np.dot(np.linalg.inv(covariance),parm_j_ps))
                F_ij[i,j] = FIJ
        F_ij_inv = np.linalg.inv(F_ij)  
        Fisher[k,:,:]= F_ij
        Fisher_Inv[k,:,:] = F_ij_inv
    return Fisher,Fisher_Inv,samples

In [ ]:
# Get Fisher Matrix
PS_Fisher,PS_Fisher_Inv,samples = get_fisher(PS_derivs,PS_fid,nparams)


In [ ]:
Using chain consumer to plot the matrix.

In [ ]:
Fisher_Param = ['$T_{Vir}$','$R_{Max}$','$\zeta$']

from chainconsumer import ChainConsumer
fid = [pow(10,4.7),15,30]

c = ChainConsumer()
c.add_covariance(fid,PS_Fisher_Inv[-1], parameters=Fisher_Param,shade=True,name='Power Spectrum')
fig = c.plotter.plot(figsize='PAGE')
fig.set_size_inches(3 + fig.get_size_inches())  

In [ ]:
Summary_Param = ['T_{Vir}','R_{Max}','\zeta']

c.add_covariance(fid,PS_Fisher_Inv[-1], parameters=Summary_Param,shade=True,name='Power Spectrum')

c.plotter.plot_summary(errorbar=True,truth = fid,extra_parameter_spacing=6,show_names = False,parameters=Summary_Param)	

For comparisons, simply at covariances via 'c.add_covariance()' of the different statistics to compare.